In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Loading the dataset from drive and configuring cudnn


In [0]:
#Uncomment to unzip in colab (first time)
!cp "/content/drive/My Drive/dress_dataset_v2.zip" "dress_dataset_v2.zip"
!unzip -q dress_dataset_v2.zip
!rm dress_dataset_v2.zip
%cd dress_dataset_v2
!rm -r kimono
%cd ..
#cudNN
!tar -xzvf /content/drive/My\ Drive/cudnn-10.1-linux-x64-v7.6.5.32.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Loading the dataset in YOLO format

In [0]:
#import libraries
import numpy as np
import cv2
import os
import math
import re

In [0]:
#Function to load the data and bounding boxes
def load_dataset(path, classes) :
    label_enc = 0;
    for cls in classes :
        subclasses = os.listdir(path + cls + "/")
        for subclass in subclasses :
            if subclass == 'bboxes.txt' :
            # if not os.path.isdir(path + cls + "/" + subclass) :
                f = open(path + cls + "/" + "bboxes.txt", "r")
                img_path_file = open(path + cls + "/" + "images.txt", "w+")

                for line in f :
                    coordinates = []
                    c = line[-16:]
                    try :
                        #Load the Images
                        img_path = path + cls + "/" + line[:-16]
                        img_path = img_path.strip()
                        img = cv2.imread(img_path)
                        # if img is not None :
                          

                        y_ = img.shape[0]
                        x_ = img.shape[1]

                        x_scale = 224/x_
                        y_scale = 224/y_

                        #Load bounding boxes
                        for i in range(4) :
                            for coordinate in c[i*4:i*4+4].split() :
                                if coordinate.isdigit() :
                                    coordinates.append(int(coordinate))

                        #Write the path to the image text file
                        img_path_file.write('{}\n'.format(img_path))
      
                        #Write bounding boxes and labels to text file
                        img_bbox_filename = re.sub('jpg$', 'txt',img_path)
                        img_file = open(img_bbox_filename, "w+")

                        #Convert to yolo format
                        x = ((coordinates[0] + coordinates[2])/2)/x_
                        width = (coordinates[2] - coordinates[0])/x_
                        y = ((coordinates[1] + coordinates[3])/2)/y_
                        height = (coordinates[3] - coordinates[1])/y_
                        #Write to the file
                        img_file.write('{} {} {} {} {}\n'.format(label_enc, x, y, width, height))
                        img_file.close()
                    except Exception as e :
                        pass

                f.close()
                img_path_file.close()
        label_enc += 1
    return


In [0]:
#Load the dataset
DATADIR = "/content/dress_dataset_v2/"
classes = os.listdir("/content/dress_dataset_v2")
load_dataset(DATADIR, classes)


# Compiling darknet

In [0]:
# !rm -r "/content/darknet"
#Setting up yolo
!git clone https://github.com/AJ-RR/darknet.git
%cd darknet
!make
%cd ..

# Balancing the dataset

In [0]:
#function to balance the dataset
def balance_dataset(path, classes) :
  minimum = 9999999999
  for cls in classes :
    # subclasses = os.listdir(path + cls + "/")
    f = open(path + cls + "/" + "images.txt")
    counter = 0
    contents = f.read()
    lines = contents.split("\n")
    for i in lines :
      if i :
        counter += 1
    if counter <= minimum :
      minimum = counter
    counter = 0
    f.close()
  count = 0
  lines = []
  # final_file = open("imgs.txt", "w+")
  for cls in classes :
    # subclasses = os.listdir(path + cls + "/")
    final_file = open(path + cls +"/" + "processed_images.txt", "w+")
    with open(path + cls + "/" + "images.txt") as f :
      lines = random.sample(f.readlines(), minimum)
      for line in lines :
        final_file.write('{}'.format(line))
        count += 1
    final_file.close()
  print("Total number of images per class = ", count/5)
  
  return count/5

In [0]:
import random
total_perclass = balance_dataset("/content/dress_dataset_v2/", os.listdir("/content/dress_dataset_v2"))


In [0]:
#function to split the dataset
def split_train_test(filename, total_imgs, test_ratio) :
  test_indices = random.sample(range(0, total_imgs), round(total_imgs*test_ratio))
  with open(filename, "r") as f :
    all_lines = f.readlines()
    test_lines = [all_lines[i] for i in test_indices] 
    all_lines = [i for j, i in enumerate(all_lines) if j not in test_indices]
    train_file = open("/content/darknet/data/dress_train.txt", "a+")
    for line in all_lines :
      if line != '\n' :
        train_file.write('{}'.format(line))
    train_file.close()
    test_file = open("/content/darknet/data/dress_test.txt", "a+")
    for line in test_lines :
      if line != '\n' :
        test_file.write('{}'.format(line))
    test_file.close()
  return 

def split_dataset(total_perclass, test_ratio, classes) :
  for cls in classes :
    split_train_test("/content/dress_dataset_v2/" + cls + "/processed_images.txt", total_perclass, test_ratio)

In [0]:
total_perclass = int(total_perclass)
split_dataset(total_perclass, 0.3, classes)  

In [0]:
# # #Get pre-trained weights
# !cp "/content/drive/My Drive/yolo/yolo-tiny-backup/dress_best.weights" "/content/darknet/dress_best.weights"
# !cp "/content/dress.names" "/content/darknet/data/dress.names"
!cp "/content/drive/My Drive/yolov3-tiny.weights" "/content/darknet/yolov3-tiny.weights"
%cd /content/darknet
!./darknet partial cfg/dress.cfg yolov3-tiny.weights yolov3-tiny.conv.15 15

# Calculate Anchors

In [0]:
# #Calculating anchors
# %cd /content/darknet
# !./darknet detector calc_anchors data/dress.data -num_of_clusters 6 -width 416 -height 416 -show

# Train the model

In [0]:
%%capture capt
!./darknet detector train data/dress.data cfg/dress.cfg -dont_show -map > yolo_tiny.log              

# Test the model

In [0]:
%cd /content/darknet/
!./darknet detector map data/dress.data cfg/dress.cfg dress_best.weights 
